In [145]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [146]:
base_url = 'https://www.genie.co.kr/chart/top200'
main_url = '?ditc=D&ymd=20230918&hh=16&rtm=Y&'
url = f'{base_url}{main_url}?pg=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [147]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers = header)
res.text[:500]

'\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t\n\n\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<!--[if lte IE 6]><html dir="ltr" class="no-js ie6"><![endif]-->\n<!--[if lte IE 7]><html dir="ltr" class="no-js ie7"><![endif]-->\n<!--[if IE 8]><html dir="ltr" class="no-js ie8"><![endif]-->\n<!--[if IE 9]><html dir="ltr" class="no-js ie9"><![endif]-->\n<!--[if (gte IE 10)|!(IE)]><!--><html dir="ltr" class="no-js"><!--<![endif]-->\n<title>음악, 그리고 설레임 - 지니</title>\n<meta charset="utf-8'

In [148]:
soup = BeautifulSoup(res.text, 'html.parser')

In [169]:
from selenium import webdriver

driver = webdriver.Chrome()     # PATH 설정 할 것 (chromedriver.exe가 위치해있는곳을 path 설정해야함)
driver.get(url)

In [176]:
# 크롬을 사용하여 불러오면 현재 지니에 적용된 시간을 불러올 수 있음
soup = BeautifulSoup(driver.page_source, 'html.parser')
date = soup.select_one('#inc_date').get_text().strip().replace('.', '')
time = soup.select_one('#inc_time').get_text().strip()[:2]
date, time

('20230918', '17')

In [167]:
# Beautifulsoup 에서는 예전 시간만 불러옴
date_str = soup.select_one('#inc_date').get_text().strip() + \
            soup.select_one('#inc_time').get_text().strip()
date_str = date_str.replace('.','').replace(':','')
date_str

'201303061200'

In [150]:
from datetime import datetime, timedelta

now = datetime.now()
date = now.strftime('%Y%m%d')
hour = now.strftime('%H')

In [151]:
trs = soup.select('tr.list')
len(trs)

50

In [152]:
tr = trs[0]

for page_num in range(1, 5):
    sub_href = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={date}&hh={time}&rtm=Y&pg={page_num}'
    sub_res = requests.get(url)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

sub_href

'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20230918&hh=16&rtm=Y&pg=4'

In [154]:
tr = trs[0]
rank = int(tr.select_one('.number').get_text().replace(' ', '').split('\n')[0])
rank

1

In [155]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'Love Lee'

In [156]:
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

'AKMU (악뮤)'

In [157]:
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'Love Lee'

In [177]:
data = []
from datetime import datetime, timedelta

for page_num in range(1, 5):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    date = soup.select_one('#inc_date').get_text().strip().replace('.', '')
    time = soup.select_one('#inc_time').get_text().strip()[:2]
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={date}&hh={time}&rtm=Y&pg={page_num}'
    res = requests.get(url, headers = header)
    soup = BeautifulSoup(res.text, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = int(tr.select_one('.number').get_text().replace(' ', '').split('\n')[0])
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()

        data.append({'순위': rank, '제목': title, '아티스트': artist, '앨범': album})

In [178]:
df = pd.DataFrame(data)
df

,순위,제목,아티스트,앨범
0,1,Love Lee,AKMU (악뮤),Love Lee
1,2,Smoke (Prod. by Dynamicduo & Padi),다이나믹 듀오 & 이영지,스트릿 우먼 파이터2 (SWF2) 계급미션
2,3,후라이의 꿈,AKMU (악뮤),Love Lee
3,4,ETA,NewJeans,NewJeans 2nd EP 'Get Up'
4,5,Seven (Feat. Latto) (Clean Ver.),정국,Seven (Clean Ver.)
...,...,...,...,...
195,196,"10,000 Hours",Dan ＋ Shay & Justin Bieber,Good Things
196,197,Lonely,2NE1,Lonely
197,198,나의 바람 (Wind And Wish),비투비,WIND AND WISH
198,199,Love Love Love (Feat. Yoong Jin of Casker)),에픽하이 (EPIK HIGH),Remapping The Human Soul
